<a href="https://colab.research.google.com/github/likeshd/case_studies_and_projects/blob/main/nlp_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mysql-connector-python
import pandas as pd
from datetime import datetime

In [ ]:
!pip install rembg==2.0.38
!pip install Pylette

In [ ]:
import mysql.connector

In [ ]:
def db_connect(conf):
    try:
        return mysql.connector.connect(**conf)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)

In [ ]:
def get_db_data(cnx, query):
    if cnx and cnx.is_connected():
        rows = []
        with cnx.cursor() as cursor:
            cursor.execute(query)
            rows = cursor.fetchall()
        cnx.close()
        return rows
    else:
        print("Could not connect to DB")

In [ ]:
def preprocess(text):
    import re
    text = text if type(text)!= float else ''
    text = text if len(text)!=0 else ''
    text = text.lower()
    text=re.sub('<[^<]+?>', ' ', text)
    text=re.sub('t-shirt', 'tshirt', text)
    text=re.sub('[^a-zA-Z0-9]+', ' ', text)
    return text

def tag_attributes(x):
    import nltk
    nltk.download('stopwords')
    from nltk.corpus import stopwords
    from collections import OrderedDict
    stop_words = set(stopwords.words('english'))
    x = preprocess(x)
    x = x.split()
    x = [word for word in x if word not in stop_words]
    # x = [str(word) for word in x if type(word)== int or len(word)>=2]
    x = list(OrderedDict.fromkeys(x))
    return x


In [ ]:
def get_product_color(img_url):
    def remove_background_from_url(url):
        import requests
        import io
        from io import BytesIO
        from PIL import Image
        from rembg import remove
        import base64
        if len(url)<=0:
            return None
        else:
            response = requests.get(url)
            try:
                input_image = Image.open(BytesIO(response.content))
                output_image = remove(input_image)
                output_buffer = BytesIO()
                output_image.save(output_buffer, format="PNG")
                output_image_data = output_buffer.getvalue()
                base64_image = base64.b64encode(output_image_data).decode('utf-8')
                image_bytes = io.BytesIO(base64.b64decode(base64_image))
                image = Image.open(image_bytes)
            except Exception as e:
                # image = image_bytes = None
                image_bytes = None
            return image_bytes

    def closest_colour(requested_colour):
        import webcolors
        min_colours = {}
        for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
            r_c, g_c, b_c = webcolors.hex_to_rgb(key)
            rd = (r_c - requested_colour[0]) ** 2
            gd = (g_c - requested_colour[1]) ** 2
            bd = (b_c - requested_colour[2]) ** 2
            min_colours[(rd + gd + bd)] = name
        return min_colours[min(min_colours.keys())]

    def get_colour_name(requested_colour):
        try:
            import webcolors
            # closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
            closest_name = webcolors.rgb_to_name(requested_colour)
        except ValueError:
            closest_name = closest_colour(requested_colour)
            # actual_name = None
        # return actual_name, closest_name
        return closest_name

    image_bytes = remove_background_from_url(img_url)

    if image_bytes!=None:
        # display(image)
        from Pylette import extract_colors
        size = 3 # Size 3 is fixed, select the patelle[1](second color) as the product color
        palette = extract_colors(image=image_bytes, palette_size=size, resize=True)
        # palette.display(save_to_file=False)

        requested_colour = palette[1].rgb
        closest_color = get_colour_name(requested_colour)
        # print("Actual colour name:", actual_color, ", closest colour name:", closest_color)
        # print(closest_color,'\n')
        return closest_color
    else:
        return "None"

In [ ]:
def create_tags_data():
    # from Similar_Products import db_connect, get_db_data, DB_CONFIG
    # from auto_tags import generate_tags_yak   e
    cnx = db_connect(DB_CONFIG)
    query = 'SELECT ProductId, ProductName, ImageURL, Brand, Category, TagAttributes FROM Product where user_created ="0c5c9299-8f80-47f1-8467-f9ce7834c458" '

    products = get_db_data(cnx, query)
    import pandas as pd
    productData = pd.DataFrame(products, columns=['ProductId', 'ProductName', 'ImageURL', 'Brand', 'Category', 'TagAttributes'])
    productData.fillna('', inplace=True)

    ###### Text Tag Generation ######
    productData['NER_Text'] = productData["ProductName"]+ ' ' +productData["Category"]+ ' ' +productData['Brand']+ ' ' +productData['TagAttributes']
    productData['NER_Text'] = productData['NER_Text'].apply(lambda x: preprocess(x))

    # productData['Product_Tags'] = productData['NER_Text'].apply(lambda x: generate_tags_yake(x))
    productData['Product_Tags'] = productData['NER_Text'].apply(lambda x: tag_attributes(x))

    ###### Product Color Fetching ######
    productData['ProductColor'] = productData['ImageURL'].apply(lambda url: get_product_color(str(url)))
    productData['ProductColor'] = productData['ProductColor'].apply(lambda x: [] if x == "None" else [x])
    productData['Product_Tags'] = productData['Product_Tags'] + productData['ProductColor']

    import json
    productData['Product_Tags'] = productData['Product_Tags'].apply(lambda x: json.dumps(x))
    productData1 = productData.drop(columns=['NER_Text', 'ImageURL', 'ProductColor'], axis=1)



    return productData1

In [ ]:
x = create_tags_data()
x

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /ro

,ProductId,ProductName,Brand,Category,TagAttributes,Product_Tags
0,product1,Product 1,,Camera,,"[""product"", ""1"", ""camera"", ""darkslategray""]"
1,test_product_id,Chef's Apron - new,,All Home,"APRON02Chef's ApronChef's,Apron,Home items, Ho...","[""chef"", ""apron"", ""new"", ""home"", ""apron02chef""..."
2,2495cc4903e8ebf72b548e7ed57f2495,Cool Gear® Arise Wave - 28 Ozzz,NUSH,"Sport Bottles, Bottles","Sport Bottles, Bottle, Plastic bottle","[""cool"", ""gear"", ""arise"", ""wave"", ""28"", ""ozzz""..."
3,2495cc4903e8ebf72b548e7ed57f2495,Cool Gear® Arise Wave - 28 Ozzz,NUSH,"Sport Bottles, Bottles","Sport Bottles, Bottle, Plastic bottle","[""cool"", ""gear"", ""arise"", ""wave"", ""28"", ""ozzz""..."
4,4982b2b1a76a85a85c9adc8b4b2d523a,Cool Gear® Arise Wave - 28 Ozzz,NUSH,"Sport Bottles, Bottles","Sport Bottles, Bottle, Plastic bottle","[""cool"", ""gear"", ""arise"", ""wave"", ""28"", ""ozzz""..."
5,7c4b0cf45bc3a84e1d0cc37c08a248,Women-Mellow-Beanie-Blue-Cap,Roxy,Sport wear,blue Cap1,"[""women"", ""mellow"", ""beanie"", ""blue"", ""cap"", ""..."
6,7c4b0cf45bc3a84e1d0cc37c08a248,Rxy-Women-Mellow-Beanie-Blue-Cap,Roxy,Sport wear,blue Cap1,"[""rxy"", ""women"", ""mellow"", ""beanie"", ""blue"", ""..."
7,7c4b0cf45bc3a84e1d0cc37c08a248,Rxy-Women-Mellow-Beanie-Blue-Cap,Roxy,Sport wear,blue Cap1,"[""rxy"", ""women"", ""mellow"", ""beanie"", ""blue"", ""..."
8,102288A3-9FAD-4BF4-A186-01C7F9FCE950,Red Cotton Paper – handmade eco friendly paper,Bluecat Paper,Other,Other,"[""red"", ""cotton"", ""paper"", ""handmade"", ""eco"", ..."
9,e13f9b7d-1f53-45e4-a2b8-ff0c456e3b67,Red Cotton Paper – handmade eco friendly paper,Bluecat Paper,Other,"Eco-friendly, Handmade","[""red"", ""cotton"", ""paper"", ""handmade"", ""eco"", ..."


In [ ]:
def Get_SQL_Engine():
    from sqlalchemy import create_engine
    engine = create_engine(f"mysql+mysqlconnector://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}/{DB_CONFIG['database']}")
    return engine


In [ ]:
engine = Get_SQL_Engine()

In [ ]:

x.to_sql('ProductTags', con=engine, if_exists='replace', index=False)

19